# Project : Random Acts of Pizza

Overview:


In [3]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import nltk
import json
import pandas as pd

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

from subprocess import check_output

Print the request text and the outcome for 5 requests from the training set

In [30]:
train_json_array = np.array(pd.read_json('train.json', orient='columns'))

data_set = train_json_array[:,6]
data_labels = train_json_array[:,22]

train_data = data_set[:data_set.shape[0]/2]
train_outcomes = data_labels[:data_labels.shape[0]/2]
train_labels = np.where(train_outcomes==True, 1, 0)

dev_data = data_set[(data_set.shape[0]/2)+1:]
dev_outcomes = data_labels[(data_labels.shape[0]/2)+1:]
dev_labels = np.where(dev_outcomes==True, 1, 0)

print('There are ',train_json_array.shape[0], ' observations and ', train_json_array.shape[1], ' features\n')

yescount = 0;
nocount = 0;
for i in range(0, train_json_array.shape[0]):
    if yescount > 3 and nocount > 3:
        break 
    if train_json_array[i][22] == False and nocount < 4:
        print(i, '. Outcome: ',train_json_array[i][22],'\n', train_json_array[i][6],'\n')
        nocount += 1
    elif train_json_array[i][22] == True and yescount < 4:
        print(i, '. ',train_json_array[i][22],'\n', train_json_array[i][6],'\n')
        yescount += 1
    else:
        continue

There are  4040  observations and  32  features

0 . Outcome:  False 
 Hi I am in need of food for my 4 children we are a military family that has really hit hard times and we have exahusted all means of help just to be able to feed my family and make it through another night is all i ask i know our blessing is coming so whatever u can find in your heart to give is greatly appreciated 

1 . Outcome:  False 
 I spent the last money I had on gas today. Im broke until next Thursday :( 

2 . Outcome:  False 
 My girlfriend decided it would be a good idea to get off at Perth bus station when she was coming to visit me and has since had to spend all her money on a taxi to get to me here in Dundee. Any chance some kind soul would get us some pizza since we don't have any cash anymore? 

3 . Outcome:  False 
 It's cold, I'n hungry, and to be completely honest I'm broke. My mum said we're having leftovers for dinner. A random pizza arriving would be nice.

Edit: We had leftovers. 

5 .  True 
 

/Users/jayashreeraman/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jayashreeraman/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jayashreeraman/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:10: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/jayashreeraman/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:11: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [39]:
tfv = TfidfVectorizer(analyzer='word', max_df=0.5, ngram_range=(1,3)) 

X_train = tfv.fit_transform(train_data)
X_dev = tfv.transform(dev_data)
print(X_train.shape, train_labels.shape)

clf = LogisticRegression(penalty='l2',C=100)
clf.fit(X_train, train_labels)
pred_labels = clf.predict(X_dev)
print(pred_labels.shape)
target_names = ['Got pizza', 'No pizza']
print(classification_report(dev_labels, pred_labels, target_names=target_names))

(2020, 187375) (2020,)
(2019,)
             precision    recall  f1-score   support

  Got pizza       0.77      0.96      0.85      1531
   No pizza       0.40      0.08      0.14       488

avg / total       0.68      0.75      0.68      2019

